## Demo for Random TODOList

- 我的 Random 应该加在 GitHub Action 上面, 每个月自动化触发

## Refer

- https://pypi.org/project/py-trello/
- https://networkx.org/documentation/stable/tutorial.html

['config.ini']

## Connect to Trello

In [43]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import trello

def patched_fetch_json(self,
                       uri_path,
                       http_method='GET',
                       headers=None,
                       query_params=None,
                       post_args=None,
                       files=None):
    """ Fetch some JSON from Trello """

    # explicit values here to avoid mutable default values
    if headers is None:
        headers = {}
    if query_params is None:
        query_params = {}
    if post_args is None:
        post_args = {}

    # if files specified, we don't want any data
    data = None
    if files is None and post_args != {}:
        data = json.dumps(post_args)

    # set content type and accept headers to handle JSON
    if http_method in ("POST", "PUT", "DELETE") and not files:
        headers['Content-Type'] = 'application/json; charset=utf-8'

    headers['Accept'] = 'application/json'

    # construct the full URL without query parameters
    if uri_path[0] == '/':
        uri_path = uri_path[1:]
    url = 'https://api.trello.com/1/%s' % uri_path

    if self.oauth is None:
        query_params['key'] = self.api_key
        query_params['token'] = self.api_secret

    # perform the HTTP requests, if possible uses OAuth authentication
    response = self.http_service.request(http_method, url, params=query_params,
                                            headers=headers, data=data,
                                            auth=self.oauth, files=files,
                                            proxies=self.proxies)

    if response.status_code == 401:
        raise trello.Unauthorized("%s at %s" % (response.text, url), response)
    if response.status_code != 200:
        raise trello.ResourceUnavailable("%s at %s" % (response.text, url), response)

    return response.json()

trello.TrelloClient.fetch_json = patched_fetch_json

trello_client = trello.TrelloClient(
    api_key=config["defaut"]["api_key"],
    api_secret=config["defaut"]["api_secret"],
    token=config["defaut"]["token"]
)
all_boards = trello_client.list_boards()
last_board = all_boards[0]
print(last_board.name)
print(all_boards[0].list_lists()[3].id)

# https://github.com/sarumont/py-trello/issues/373

Hi-Kanban
631058bee13e0d048d72c450


In [44]:
import pandas as pd
def map_to(x):
    tags = " ".join([ "#" + label.name.replace(" ", "_") for label in x.labels ])
    weight = 10
    if "#high_priority" in tags:
        weight = 30

    return { 
        "title": x.name, 
        "tags": tags,
        "weight": weight, 
        "short_url": x.shortUrl,
        "uuid":x.id
    }

todo_list = last_board.get_list("631058bee13e0d048d72c450")
l = pd.DataFrame(list(map(map_to, todo_list.list_cards())))
l

,title,tags,weight,short_url,uuid
0,Cyberpunk 2077 | 赛博朋克 2077 | 估计是在 xbox 上连带着 DL...,#game,10,https://trello.com/c/Po8NMTCj,62b5786011dfb05af24da9fb
1,Celeste | 蔚蓝 | 自虐一段时间，来让自己感受生活的魅力？,#game,10,https://trello.com/c/YwcfcFQp,65e1a08b895fa2624c29b66a
2,Forza Horizon 5 | 简单开一下车吧,#game,10,https://trello.com/c/1zSvVToS,62b57ab6aa6fbe705a821a1d
3,写作知识补充，阅读之前买的那些关于写作的书,#learn #practice,10,https://trello.com/c/YuvRSj8g,62b57ce1f075270385b0b56b
4,Prompt Engineering | 稍微做一个简单的 embedding search...,#research #portfolios #high_priority,30,https://trello.com/c/0Y0q6PVo,648fcca93ff71a2867b44ad6
5,Learning English by GPT | 学学英语 结合 GPT | 每天读 Ha...,#practice #learn,10,https://trello.com/c/ujpDHZne,60df148099c8132309a50b88
6,考古 | 关于互联网技术发展史的编写 | 先从收集各种资料开始,#research,10,https://trello.com/c/RRfH1hHi,62bf93de78517a87c2d1e106
7,CS50x | 补一下 CS 的课程 | 先按着节奏过第一个月的教程，再看中间结果决定后面是...,#learn,10,https://trello.com/c/027F4vUz,65b1cf323269d26c49e9d9ce
8,补一下历史知识，尤其是近代史,#learn,10,https://trello.com/c/HfGAxpzJ,66684dc745cacad96d198e6b
9,Learn Japanese | 学学日语 | 通关生肉的 《Muv-Luv》吧，顺便过一下...,#practice #learn,10,https://trello.com/c/iShDJjft,639038e5051f3101a392a282


In [45]:
import pandas as pd
history = pd.read_csv('table.csv')
history

,Date,Title,Tags
0,2023-06,VA-11 Hall-A: Cyberpunk Bartender Action,Knowledge
1,2023-07,Apple Game Porting Toolkit Research,Knowledge
2,2023-08,The Legend of Zelda: Breath of the Wild - nira...,ACG
3,2023-09,The Legend of Zelda: Breath of the Wild - nira...,ACG
4,2023-10,Personal Information Flow Plan - 个人信息流规划,Knowledge
5,2023-11,The Legend of Zelda: Breath of the Wild - nira...,ACG
6,2023-12,Nothing,NaN
7,2024-01,Take photos to document life - 记录生活需要有照片 - 一天拍...,#practice #elegant
8,2024-02,Building an ACG Knowledge Base - 搭建 ACG 方面的知识库...,#research #portfolios #high_priority
9,2024-03,Building an ACG Knowledge Base - 搭建 ACG 方面的知识库...,#research #portfolios #high_priority


## Connect to telegram

In [46]:
import requests

channel_id = config["telegram"]["channel_id"]
bot_token = config["telegram"]["token"]
base_url = f"https://telegram.padparadsayc.com/bot{bot_token}"

def send_message_and_pin(text):
    send_message_url = f"{base_url}/sendMessage"
    message_data = {
        "chat_id": channel_id,
        "text": text,
        "parse_mode": "HTML"
    }

    # 发送消息
    response = requests.post(send_message_url, data=message_data)
    print(response.json())
    message_id = response.json()['result']['message_id']

    # 置顶消息
    pin_message_url = f"{base_url}/pinChatMessage"
    pin_data = {
        "chat_id": channel_id,
        "message_id": message_id
    }
    return requests.post(pin_message_url, data=pin_data)

In [6]:
# data = [
#     ['2023-07', '最近做的工作跟系统底层打交道的情况实在是不少，需要补一下一些虚拟化方面的知识', 'Knowledge'],
# ]

# df = pd.DataFrame(data, columns=['Date', 'Title', 'Category'])
# df.to_csv('table.csv', index=False)

## Random Now ~~

In [47]:
import random, datetime, time

user_input = input("确定要继续运行吗？(Y/n): ")    

def rs():
    return random.choices(
        population=list(l.index),
        weights=list(l.weight),
        k=1
    )

e = datetime.datetime.now()
month = (datetime.datetime.today().month) % 12

for i in range(10):
    time.sleep(3) # Delay for 3 seconds.
    random_id = rs()[0]
    if i < 9:
        print("不是: " + str(l.loc[random_id]['title']))
        
msg = "{month}月决定就是: <b>{title}</b> \n\n🌟🎉 恭喜恭喜!!! ({time} <a href=\"{url}\">传送门</a>)\n{tags} #random_todolist".format(
    month=month,
    url=l.loc[random_id]['short_url'],
    title=l.loc[random_id]['title'],
    tags=l.loc[random_id]['tags'], 
    time=e.strftime("%Y-%m-%d")
)

print("\n" + msg)
send_message_and_pin(msg)

确定要继续运行吗？(Y/n):  


不是: Prompt Engineering | 稍微做一个简单的 embedding search 给奈亚子
不是: 写作知识补充，阅读之前买的那些关于写作的书
不是: 打字练习 | 一周五次，将打字的速度评分发到 Channel 上
不是: CS50x | 补一下 CS 的课程 | 先按着节奏过第一个月的教程，再看中间结果决定后面是否继续（共三个月）
不是: Celeste | 蔚蓝 | 自虐一段时间，来让自己感受生活的魅力？
不是: 补一下设计模式的书，拿之前的一众 TODO Issue 来实践一下
不是: Math | 数学基础 | 例如可以每天看一下 3Blue1Brown 的视频
不是: Drawing Skills Training | 画画技能培训 | 每天展示一点练习进度，目标到最后可以改一改小 ni 的像素画，弄不同的 emoji
不是: 补一下历史知识，尤其是近代史

6月决定就是: <b>Math | 数学基础 | 例如可以每天看一下 3Blue1Brown 的视频</b> 

🌟🎉 恭喜恭喜!!! (2024-06-12 <a href="https://trello.com/c/mo8vDwGw">传送门</a>)
#learn #random_todolist
{'ok': True, 'result': {'message_id': 472, 'sender_chat': {'id': -1002019451870, 'title': 'Tomoko RD', 'username': 'tomoko_channel', 'type': 'channel'}, 'chat': {'id': -1002019451870, 'title': 'Tomoko RD', 'username': 'tomoko_channel', 'type': 'channel'}, 'date': 1718172438, 'text': '6月决定就是: Math | 数学基础 | 例如可以每天看一下 3Blue1Brown 的视频 \n\n🌟🎉 恭喜恭喜!!! (2024-06-12 传送门)\n#learn #random_todolist', 'entities': [{'offset': 8, 'length': 39, 'type': 'bold

<Response [200]>

In [52]:
## Sync to ...

import json

for card in todo_list.list_cards():
    if(card.id == l.loc[random_id]['uuid']):
        card.change_list("6024cd248ecb43309b5eb2d0")

from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
history = pd.read_csv('table.csv')

if len(history) == 0:
    current_date = datetime.now().strftime('%Y-%m')
else:
    previous_date = history.iloc[-1]['Date']
    previous_date = datetime.strptime(previous_date, '%Y-%m')
    current_date = previous_date + relativedelta(months=1)
    current_date = current_date.strftime('%Y-%m')

new_row = {'Date': current_date, 'Title': l.loc[random_id]['title'].replace("|", "-"), 'Tags': l.loc[random_id]['tags']}
new_df = pd.DataFrame(new_row, index=[0])

history = pd.concat([history, new_df], ignore_index=True)
markdown_table = history.to_markdown(index=False)
history

# 更新 README 文档
readme_path = 'README.md'

with open(readme_path, 'r') as file:
    readme_content = file.read()

# 找到要替换的标记，例如 <!-- TABLE_START --> 和 <!-- TABLE_END -->
table_start = '<!-- TABLE_START -->\n'
table_end = '\n<!-- TABLE_END -->'

# 替换标记位置的内容为 Markdown 表格
regex = f"{re.escape(table_start)}(.*?)\n{re.escape(table_end)}"
updated_readme_content = re.sub(regex, f"{table_start}\n{markdown_table}\n{table_end}", readme_content, flags=re.DOTALL)
updated_readme_content

# 写入更新后的 README 文件
with open(readme_path, 'w') as file:
    file.write(updated_readme_content)

print("README 文件已更新！")
history.to_csv('table.csv', index=False)
history

README 文件已更新！


,Date,Title,Tags
0,2023-06,VA-11 Hall-A: Cyberpunk Bartender Action,Knowledge
1,2023-07,Apple Game Porting Toolkit Research,Knowledge
2,2023-08,The Legend of Zelda: Breath of the Wild - nira...,ACG
3,2023-09,The Legend of Zelda: Breath of the Wild - nira...,ACG
4,2023-10,Personal Information Flow Plan - 个人信息流规划,Knowledge
5,2023-11,The Legend of Zelda: Breath of the Wild - nira...,ACG
6,2023-12,Nothing,NaN
7,2024-01,Take photos to document life - 记录生活需要有照片 - 一天拍...,#practice #elegant
8,2024-02,Building an ACG Knowledge Base - 搭建 ACG 方面的知识库...,#research #portfolios #high_priority
9,2024-03,Building an ACG Knowledge Base - 搭建 ACG 方面的知识库...,#research #portfolios #high_priority
